In [21]:
%load_ext autoreload
%autoreload 2

In [22]:
import pandas as pd
# import numpy as np

In [23]:
database_type = "/full"

The `patient` table includes general information about the patient admissions (for example, demographics, admission and discharge details). 
See: http://eicu-crd.mit.edu/eicutables/patient/

In [24]:
# loads patient table
patient = pd.read_csv('../eICU' + database_type + '/patient.csv', nrows=999999) 
patient = patient[['patientunitstayid', 'admissionweight']].set_index('patientunitstayid')
patient

,admissionweight
patientunitstayid,
141168,84.3
141178,54.4
141179,NaN
141194,73.9
141196,NaN
...,...
3353235,90.0
3353237,78.4
3353251,102.0


The `vitalperiodic` table comprises data that is consistently interfaced from bedside vital signs monitors into eCareManager. 
Data are generally interfaced as 1 minute averages, and archived into the `vitalperiodic` table as 5 minute median values. 
For more detail, see: http://eicu-crd.mit.edu/eicutables/vitalPeriodic/

In [25]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv', nrows=999999)
columns = ['patientunitstayid', 'observationoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic']
vitalperiodic = vitalperiodic[columns]
vitalperiodic = vitalperiodic.sort_values(by='observationoffset')
vitalperiodic

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic
653065,148349,-1445,NaN,60.0,20.0,NaN
653071,148349,-1440,NaN,60.0,21.0,NaN
790455,150049,-1435,NaN,96.0,19.0,NaN
479374,146418,-1435,NaN,71.0,23.0,NaN
653145,148349,-1435,NaN,60.0,21.0,NaN
...,...,...,...,...,...,...
273524,144297,71778,NaN,72.0,33.0,NaN
273140,144297,71783,NaN,72.0,27.0,NaN
273956,144297,71788,NaN,72.0,34.0,NaN
275114,144297,71793,NaN,72.0,33.0,NaN


The `lab` table provides Laboratory tests that have have been mapped to a standard set of measurements. 
Unmapped measurements are recorded in the customLab table.
See: http://eicu-crd.mit.edu/eicutables/lab/

In [26]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv', nrows=999999)
columns = ['patientunitstayid', 'labresultoffset', 'labname', 'labresult']
lab = lab[columns]
# lab = lab.sort_values(by='labresultoffset')
lab

,patientunitstayid,labresultoffset,labname,labresult
0,141168,2026,fibrinogen,177.0
1,141168,1133,PT - INR,2.5
2,141168,2026,magnesium,2.0
3,141168,1133,PT,26.6
4,141168,2141,pH,7.2
...,...,...,...,...
999994,180812,7399,bedside glucose,207.0
999995,180812,11983,sodium,140.0
999996,180812,418,PT,53.0
999997,180812,418,sodium,142.0


In [27]:
#converting labname 'urinary creatinine to 'creatinine'
lab.loc[lab['labname'].str.contains("urinary creatinine"), 'labname'] = "creatinine"

# Only keeping rows where labname is 'WBC x 1000', 'lactate', 'creatinine'
lab_name = ['WBC x 1000', 'lactate', 'creatinine']
lab = lab[lab.labname.isin(lab_name)]

# duplicating labs to transpose labresult and labname
labc = lab.copy()
labw = lab.copy()
labl = lab.copy()

labc = labc.loc[(labc['labname'].str.contains("creatinine"))]
labw = labw.loc[(labw['labname'].str.contains("WBC x 1000"))]
labl = labl.loc[(labl['labname'].str.contains("lactate"))]

In [28]:
# Transpose labresults and lab name to cols
labc['creatinine'] = labc.apply(lambda x: x['labresult'] if x['labname'] == 'creatinine' else -1, axis=1)
labw['wbcx1000'] = labw.apply(lambda x: x['labresult'] if x['labname'] == 'WBC x 1000' else -1, axis=1)
labl['lactate'] = labl.apply(lambda x: x['labresult'] if x['labname'] == 'lactate' else -1, axis=1)

labc = labc.drop(['labname', 'labresult'], axis = 1)
labw = labw.drop(['labname', 'labresult'], axis = 1)
labl = labl.drop(['labname', 'labresult'], axis = 1)

# merge lab results dataframes
finallab = labc.merge(labw, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
finallab = finallab.merge(labl, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
finallab.drop_duplicates()
finallab

,patientunitstayid,labresultoffset,creatinine,wbcx1000,lactate
0,141168,1701,173.12,NaN,NaN
1,141168,2026,2.95,19.8,12.2
2,141168,1133,2.30,14.7,NaN
3,141168,516,1.95,9.8,NaN
4,141178,-280,0.70,7.6,NaN
...,...,...,...,...,...
30639,180812,10778,1.32,NaN,NaN
30640,180812,4837,1.25,NaN,NaN
30641,180812,13693,1.33,NaN,NaN
30642,180812,14858,1.30,NaN,NaN


The `intakeOutput` table provides Intake and output recorded for patients entered from the nursing flowsheet.
See: http://eicu-crd.mit.edu/eicutables/intakeOutput/

In [29]:
intakeOutput = pd.read_csv('../eICU' + database_type + '/intakeOutput.csv', nrows=999999)
columns = ['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'celllabel']
intakeOutput = intakeOutput[columns]
intakeOutput.sort_values(by='intakeoutputoffset')
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutputUrine = pd.DataFrame.merge(patient, intakeOutputUrine, on='patientunitstayid')
intakeOutputUrine

,patientunitstayid,admissionweight,intakeoutputoffset,intaketotal,outputtotal,celllabel
0,141179,NaN,1420,0.0,700.0,Urine
1,141179,NaN,933,0.0,500.0,Urine
2,141179,NaN,1703,0.0,1000.0,Urine
3,141179,NaN,678,0.0,700.0,Urine
4,141194,73.90,12201,290.4,100.0,Urine
...,...,...,...,...,...,...
207342,425009,65.80,11046,0.0,300.0,Urine
207343,425010,65.80,1358,0.0,150.0,Urine
207344,425072,93.90,28,0.0,350.0,Urine
207345,425072,93.90,1108,0.0,0.0,Urine


In [30]:
intakeOutputUrine['admissionweight'].fillna(intakeOutputUrine['admissionweight'].mean(), inplace=True)
intakeOutputUrine['urineoutputbyweight'] = intakeOutputUrine.apply(lambda x: x['outputtotal'] / x['admissionweight'], axis=1)
intakeOutputUrine = intakeOutputUrine[['patientunitstayid','intakeoutputoffset', 'urineoutputbyweight']]
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141179,1420,7.851544
1,141179,933,5.608246
2,141179,1703,11.216491
3,141179,678,7.851544
4,141194,12201,1.353180
...,...,...,...
207342,425009,11046,4.559271
207343,425010,1358,2.279635
207344,425072,28,3.727370
207345,425072,1108,0.000000


In [168]:
# Merging Final Training Table
merge1 = pd.merge(vitalperiodic, finallab,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','labresultoffset'])
finalMerge = pd.merge(merge1, intakeOutputUrine,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','intakeoutputoffset'])

finalMerge = finalMerge[['patientunitstayid', 'observationoffset', 'labresultoffset', 'intakeoutputoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic', 'creatinine', 'wbcx1000', 'lactate', 'urineoutputbyweight']]
finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
finalMerge


,patientunitstayid,observationoffset,labresultoffset,intakeoutputoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight
34293,141168,119.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
35722,141168,124.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
36916,141168,129.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
38515,141168,134.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
40003,141168,139.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1197706,425009,NaN,NaN,11046.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.559271
1087393,425010,NaN,NaN,1358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.279635
1049726,425072,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.727370
1081281,425072,NaN,NaN,1108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [169]:
finalMerge['observationoffset'] = finalMerge.apply(lambda x: x['observationoffset'] if pd.notna(x['observationoffset']) else (x['labresultoffset'] if pd.notna(x['labresultoffset']) else x['intakeoutputoffset']), axis = 1)
finalMerge

,patientunitstayid,observationoffset,labresultoffset,intakeoutputoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight
34293,141168,119.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
35722,141168,124.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
36916,141168,129.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
38515,141168,134.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
40003,141168,139.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1197706,425009,11046.0,NaN,11046.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.559271
1087393,425010,1358.0,NaN,1358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.279635
1049726,425072,28.0,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.727370
1081281,425072,1108.0,NaN,1108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [170]:
finalMerge = finalMerge.drop(['labresultoffset', 'intakeoutputoffset'], axis = 1)
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight
34293,141168,119.0,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
35722,141168,124.0,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
36916,141168,129.0,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
38515,141168,134.0,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
40003,141168,139.0,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1197706,425009,11046.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.559271
1087393,425010,1358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.279635
1049726,425072,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.727370
1081281,425072,1108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [152]:
# vitalperiodic = vitalperiodic.sort_values(by=['observationoffset'])
# finallab = finallab.sort_values(by=['labresultoffset'])
# intakeOutputUrine = intakeOutputUrine.sort_values(by=['intakeoutputoffset'])
# merge1 = pd.merge_asof(vitalperiodic, finallab,  by='patientunitstayid', left_on=['observationoffset'], right_on = ['labresultoffset'], direction='nearest')
# finalMerge = pd.merge_asof(merge1, intakeOutputUrine,  by='patientunitstayid', left_on=['observationoffset'], right_on = ['intakeoutputoffset'], direction='nearest')

# finalMerge = finalMerge[['patientunitstayid', 'observationoffset', 'labresultoffset', 'intakeoutputoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic', 'creatinine', 'wbcx1000', 'lactate', 'urineoutputbyweight']]
# finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
# finalMerge

In [171]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))

In [172]:
ff = finalMerge.copy()
nfinalMerge = pd.DataFrame()

In [195]:
for id in idList[:51]:
    df = ff[ff['patientunitstayid'] == id]
    df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
    print(df)
    print(df['lactate'].interpolate(method='linear', limit_direction='both').shape)
    nfinalMerge = nfinalMerge.append(df)
    print("next")    

         patientunitstayid  observationoffset  temperature  heartrate  \
34293               141168              119.0          NaN      140.0   
35722               141168              124.0          NaN      140.0   
36916               141168              129.0          NaN      140.0   
38515               141168              134.0          NaN      140.0   
40003               141168              139.0          NaN      140.0   
...                    ...                ...          ...        ...   
496613              141168             2314.0          NaN      100.0   
1024705             141168              516.0          NaN      100.0   
1024706             141168             1133.0          NaN      100.0   
1024707             141168             1701.0          NaN      100.0   
1024708             141168             2026.0          NaN      100.0   

         respiration  systemicsystolic  creatinine  wbcx1000  lactate  \
34293            NaN              40.0        1.95

In [191]:
# groupeddf = finalMerge['temperature'].groupby('patientunitstayid')
# for i in groupeddf['patientunitstayid', 'temperature']:
#     if(i[0] == ['patientunitstayid', 'temperature']):    
#         print(i)
# #     print("next group")

In [165]:
nfinalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight
34293,141168,119.0,NaN,140.0,NaN,40.0,1.95,9.8,12.2,NaN
35722,141168,124.0,NaN,140.0,NaN,40.0,1.95,9.8,12.2,NaN
36916,141168,129.0,NaN,140.0,NaN,40.0,1.95,9.8,12.2,NaN
38515,141168,134.0,NaN,140.0,NaN,40.0,1.95,9.8,12.2,NaN
40003,141168,139.0,NaN,140.0,NaN,40.0,1.95,9.8,12.2,NaN
...,...,...,...,...,...,...,...,...,...,...
1191708,141548,9872.0,NaN,84.0,20.0,NaN,1.18,12.5,NaN,5.047421
1200244,141548,11606.0,NaN,84.0,20.0,NaN,1.18,12.5,NaN,5.047421
1202695,141548,12220.0,NaN,84.0,20.0,NaN,1.18,12.5,NaN,3.364947
1203730,141548,12499.0,NaN,84.0,20.0,NaN,1.18,12.5,NaN,2.804123
